In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

#print('key:', OPENAI_API_KEY)

In [5]:
from openai import OpenAI

client =  OpenAI(api_key=OPENAI_API_KEY)

In [6]:
from openai import OpenAI

client =  OpenAI(api_key=OPENAI_API_KEY)

response = client.responses.create(
    model='gpt-4.1',
    input='스타워즈 시리즈의 다스베이더의 역할에 대해 한줄로 설벙해줘'
)

In [7]:
response.output_text

'다스베이더는 스타워즈 시리즈에서 포스를 어둠의 편에 사용하며 은하 제국의 상징적인 악역으로 등장하는 캐릭터입니다.'

In [8]:
response = client.responses.create(
    model='gpt-4.1',
    instructions='당신은 영화평론가야!',
    input='스타워즈 시리즈의 다스베이더의 역할에 대해 한줄로 설명해줘'
)
print(response.output_text)

다스 베이더는 스타워즈 시리즈에서 과거의 영웅이자 타락한 제다이로, 은하 제국의 위협과 아버지로서의 비극을 상징하는 아이콘이다.


In [9]:
response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role': 'developer',
            'content': '음식에 대한 이야기 하는것을 좋아해'
        },
        {
            'role': 'user',
            'content': '오늘은 무엇을 먹을까?'
        }
    ]
)
print(response.output_text)

오늘은 특별히 어떤 음식이 땡기나요? 아니면, 간단하게 추천을 해줄까요?

예를 들어,

- **한식**: 집밥 느낌 나는 김치찌개, 제육볶음, 비빔밥
- **중식**: 짜장면, 탕수육, 마파두부
- **일식**: 스시, 카레, 규동
- **양식**: 파스타, 스테이크, 샐러드, 치킨
- **분식**: 떡볶이, 순대, 김밥, 라면

집에 있는 재료를 알려주시면 활용할 레시피도 추천드릴 수 있어요!  
혹시 기분이나 상황(혼밥, 가족과, 친구와 등)에 맞춰서도 추천해 드릴 수 있습니다 :)

조금만 더 힌트를 주시면 더 딱 맞는 메뉴를 추천해 드릴 수 있어요!


In [10]:
completion = client.chat.completions.create(
    model='gpt-4.1',
    messages= [
        {
            'role': 'user',
            'content': '스타워즈시리즈에 대해 한분장으로 설명해줘'
        }
    ]  
)


In [11]:
#dict(completion.choices[0])
completion.choices[0].message.content

'스타워즈 시리즈는 은하계의 선과 악의 대결, 제다이와 시스, 다양한 캐릭터와 종족이 등장하는 광대한 세계관 속에서 자유와 정의, 가족과 운명을 다루는 SF 판타지 영화 시리즈입니다.'

In [12]:
user_input = input('안녕하세요. 무엇이 굼금하신가요? 질문을 남겨주세요!!!')
print('질문:', user_input)

response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role': 'developer',
            'content': '너는 유치원 선생님이야. 아이들을 대하는것처럼 이야기해줘'
        },
        {
            'role': 'user',
            'content': user_input
        }
    ]
)
print(response.output_text)

질문: 
안녕~! 우리 친구 반가워요😊 오늘은 어떤 재미있는 이야기를 들려줄까? 궁금한 게 있으면 뭐든지 물어봐도 돼요! 선생님이 친절하게 설명해줄게요. 우리 다 같이 즐겁게 놀면서 배워볼까요?👋✨


In [13]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [14]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
    "required": ["latitude", "longitude"],
    "additionalProperties": False
    },
    "strict": True
}]

In [15]:
input_messages = [{ 'role': 'user', 'content':'오늘 파리 날씨 어때?'}]

# 함수 호출하는 기능
response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

In [16]:
dict(response.output[0])

{'arguments': '{"latitude":48.8566,"longitude":2.3522}',
 'call_id': 'call_0OEmbaeoLDqcTXOiuHPDeGjS',
 'name': 'get_weather',
 'type': 'function_call',
 'id': 'fc_685cd00d0f4081998d310291b6f4bd2c06ac7c96e5934b8a',
 'status': 'completed'}

In [17]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args["latitude"], args["longitude"])
print(result)

20.5


In [18]:
input_messages.append(tool_call)
input_messages.append({
    'type' : 'function_call_output',
    'call_id': tool_call.call_id,
    'output': str(result)
})

# 최종 응답
response2 = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(response2.output_text)

오늘 파리의 현재 기온은 약 20.5°C입니다. 쾌적한 날씨네요! 다른 정보가 필요하신가요?


In [19]:
from tools.basic_tools import calculate_age, convert_currency, calculate_bmi

In [20]:
fn_tools = [
    {
        "type": "function",
        "name": "calculate_age",
        "description": "입력된 생년월일(YYYY-mm-dd)로 만 나이를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "birthdate": {"type": "string", "description": "생년월일, 형식:YYYY-MM-DD"}
            },
        "required": ["birthdate"],
        "additionalProperties": False
        },
        "strict": True
    },
    {
        "type": "function",
        "name": "convert_currency",
        "description": "입력된 달러(USD)를 원화(KRW)로 변환합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {"type": "number", "description": "달러(USD) 금액"}
            },
        "required": ["amount"],
        "additionalProperties": False
        },
        "strict": True
    },
    {
        "type": "function",
        "name": "calculate_bmi",
        "description": "키(cm), 몸무게(kg) 정보를 받아서 BMI를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "height": {"type": "number", "description": "키(cm)"},
                "weight": {"type": "number", "description": "몸무게(cm)"}
            },
        "required": ["height", "weight"],
        "additionalProperties": False
        },
        "strict": True
    }
]

In [21]:
input_messages = [
    {
        "role" : "user",
        "content": "내 생일은 1990-01-01 이고, 100 달러를 가지고 있는데 원화로 변환해줘, 그리고 내 키는 190cm 이고 몸무게는 50kg 이야 BMI 점수를 알려줘"
    }
]

response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=fn_tools
)

print(response.output)
print(response.output_text)

[ResponseFunctionToolCall(arguments='{"birthdate":"1990-01-01"}', call_id='call_4Kz3UbOCOzmJNFmyH4v3CiuF', name='calculate_age', type='function_call', id='fc_685cd01066a88198a8903887aed2241705ab1123ab438b36', status='completed'), ResponseFunctionToolCall(arguments='{"amount":100}', call_id='call_jCSn5QhgIzSr3uWxwsIPALAg', name='convert_currency', type='function_call', id='fc_685cd0107d3481988506d56f880eee4105ab1123ab438b36', status='completed'), ResponseFunctionToolCall(arguments='{"height":190,"weight":50}', call_id='call_suwPRZVGnmaooTmOtVRylaA6', name='calculate_bmi', type='function_call', id='fc_685cd010945c8198bcc7e2bd7b1ace0e05ab1123ab438b36', status='completed')]



In [22]:
print(response.output[0])
print(response.output[1])
print(response.output[2])

ResponseFunctionToolCall(arguments='{"birthdate":"1990-01-01"}', call_id='call_4Kz3UbOCOzmJNFmyH4v3CiuF', name='calculate_age', type='function_call', id='fc_685cd01066a88198a8903887aed2241705ab1123ab438b36', status='completed')
ResponseFunctionToolCall(arguments='{"amount":100}', call_id='call_jCSn5QhgIzSr3uWxwsIPALAg', name='convert_currency', type='function_call', id='fc_685cd0107d3481988506d56f880eee4105ab1123ab438b36', status='completed')
ResponseFunctionToolCall(arguments='{"height":190,"weight":50}', call_id='call_suwPRZVGnmaooTmOtVRylaA6', name='calculate_bmi', type='function_call', id='fc_685cd010945c8198bcc7e2bd7b1ace0e05ab1123ab438b36', status='completed')


In [23]:
import json

if response.output :
    for tool_call in response.output:
        if tool_call.type == 'function_call':
            if tool_call.name == 'calculate_age':
                print(f'호출 도구:{tool_call.name}')
                print(f'매개변수: {tool_call.arguments}')
                # 매개변수 추출
                args = json.loads(tool_call.arguments)
                # 함수 실행 결과
                result = calculate_age(args['birthdate'])
                input_messages.append(tool_call)
                input_messages.append({
                    "type" : "function_call_output",
                    "call_id": tool_call.call_id,
                    "output": str(result)
                })
            
            
            if tool_call.name == 'convert_currency':
                print(f'호출 도구:{tool_call.name}')
                print(f'매개변수: {tool_call.arguments}')

                # 매개변수 추출
                args = json.loads(tool_call.arguments)

                # 함수 실행 결과
                result = convert_currency(args['amount'])

                input_messages.append(tool_call)
                input_messages.append({
                    "type" : "function_call_output",
                    "call_id": tool_call.call_id,
                    "output": str(result)
                })
            
            if tool_call.name == 'calculate_bmi':
                print(f'호출 도구:{tool_call.name}')
                print(f'매개변수: {tool_call.arguments}')

                # 매개변수 추출
                args = json.loads(tool_call.arguments)

                # 함수 실행 결과
                result = calculate_bmi(args['height'],args['weight'])

                input_messages.append(tool_call)
                input_messages.append({
                    "type" : "function_call_output",
                    "call_id": tool_call.call_id,
                    "output": str(result)
                })

print(input_messages)
            

호출 도구:calculate_age
매개변수: {"birthdate":"1990-01-01"}
호출 도구:convert_currency
매개변수: {"amount":100}
호출 도구:calculate_bmi
매개변수: {"height":190,"weight":50}
[{'role': 'user', 'content': '내 생일은 1990-01-01 이고, 100 달러를 가지고 있는데 원화로 변환해줘, 그리고 내 키는 190cm 이고 몸무게는 50kg 이야 BMI 점수를 알려줘'}, ResponseFunctionToolCall(arguments='{"birthdate":"1990-01-01"}', call_id='call_4Kz3UbOCOzmJNFmyH4v3CiuF', name='calculate_age', type='function_call', id='fc_685cd01066a88198a8903887aed2241705ab1123ab438b36', status='completed'), {'type': 'function_call_output', 'call_id': 'call_4Kz3UbOCOzmJNFmyH4v3CiuF', 'output': '35'}, ResponseFunctionToolCall(arguments='{"amount":100}', call_id='call_jCSn5QhgIzSr3uWxwsIPALAg', name='convert_currency', type='function_call', id='fc_685cd0107d3481988506d56f880eee4105ab1123ab438b36', status='completed'), {'type': 'function_call_output', 'call_id': 'call_jCSn5QhgIzSr3uWxwsIPALAg', 'output': '133000'}, ResponseFunctionToolCall(arguments='{"height":190,"weight":50}', call_id='call_suwPRZV

In [24]:
response_msg = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=fn_tools
)

print(response_msg.output_text)

- 만 나이는 35세입니다.
- 100달러는 약 133,000원입니다.
- 키 190cm, 몸무게 50kg 기준 BMI는 13.85로, 저체중에 해당합니다.


### File Search

In [27]:
import requests
from io import BytesIO
#from openai import OpenAI

#client = OpenAI()

def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id

In [33]:
file_id = create_file(client, './howto-sockets.pdf')

file-1TYoyaaBCuda9UZk83cnS4


In [34]:
vector_store = client.vector_stores.create(
    name='knowledge_base'
)

print(vector_store.id)

vs_685cd181f51c8191a1917d95f8c9fd1a


In [35]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)

VectorStoreFile(id='file-1TYoyaaBCuda9UZk83cnS4', created_at=1750913410, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_685cd181f51c8191a1917d95f8c9fd1a', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [36]:
result_list = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)

print(result_list)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-1TYoyaaBCuda9UZk83cnS4', created_at=1750913410, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_685cd181f51c8191a1917d95f8c9fd1a', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-1TYoyaaBCuda9UZk83cnS4', last_id='file-1TYoyaaBCuda9UZk83cnS4')


In [37]:
response = client.responses.create(
    model='gpt-4.1',
    input='파이썬 코드로 소켓 만드는 방법을 간단하게 설명해줘',
    tools=[{
        "type" : "file_search",
        "vector_store_ids" : [vector_store.id]
    }],
    tool_choice='required'
)

print(response)

Response(id='resp_685cd1836d10819ab5cf1c3cd80e595301affa5d4c0dd025', created_at=1750913411.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_685cd1841e1c819abef53887e158825901affa5d4c0dd025', queries=['파이썬 소켓 만드는 방법', '파이썬 소켓 코드 예시', '파이썬 소켓 간단 설명', 'How to make a socket in Python'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_685cd186b040819a9e4b650c8326879101affa5d4c0dd025', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-1TYoyaaBCuda9UZk83cnS4', filename='howto-sockets.pdf', index=810, type='file_citation')], text='파이썬에서 소켓을 만드는 방법을 간단하게 설명하면 다음과 같습니다.\n\n### 1. 소켓 모듈 임포트\n```python\nimport socket\n```\n\n### 2. 소켓 객체 생성\n```python\ns = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\n```\n- `AF_INET`은 IPv4를, `SOCK_STREAM`은 TCP 프로토콜을 사용한다는 뜻입니다.\n\n### 3. 서버(수신자)와 클라이언트(송신자) 구분\n\n#### (1) 클라이언트 소

In [38]:
print(dict(response))

{'id': 'resp_685cd1836d10819ab5cf1c3cd80e595301affa5d4c0dd025', 'created_at': 1750913411.0, 'error': None, 'incomplete_details': None, 'instructions': None, 'metadata': {}, 'model': 'gpt-4.1-2025-04-14', 'object': 'response', 'output': [ResponseFileSearchToolCall(id='fs_685cd1841e1c819abef53887e158825901affa5d4c0dd025', queries=['파이썬 소켓 만드는 방법', '파이썬 소켓 코드 예시', '파이썬 소켓 간단 설명', 'How to make a socket in Python'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_685cd186b040819a9e4b650c8326879101affa5d4c0dd025', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-1TYoyaaBCuda9UZk83cnS4', filename='howto-sockets.pdf', index=810, type='file_citation')], text='파이썬에서 소켓을 만드는 방법을 간단하게 설명하면 다음과 같습니다.\n\n### 1. 소켓 모듈 임포트\n```python\nimport socket\n```\n\n### 2. 소켓 객체 생성\n```python\ns = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\n```\n- `AF_INET`은 IPv4를, `SOCK_STREAM`은 TCP 프로토콜을 사용한다는 뜻입니다.\n\n### 3. 서버(수신자)와 클라이언트(송신자) 구분\

In [39]:
completion = client.chat.completions.create(
    model='gpt-4.1',
    messages=[{
        "role" : "user",
        "content" : ' 스타워즈 시리즈에 대해 한 문장으로 설명해줘'
    }]
)

print(completion.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='스타워즈 시리즈는 미래 은하계를 배경으로 선과 악의 대립, 가족의 비극과 영웅의 여정을 그린 SF 영화 프랜차이즈입니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))
